In [10]:
%pip install torch zkstats==0.1.5


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
import json
import os
from zkstats.core import (
    prover_gen_settings,
    prover_gen_proof,
    setup,
    verifier_verify,
)
from zkstats.computation import computation_to_model

cwd = os.getcwd()

# FIXME: fill this in with the path to your data
data_path = f"{cwd}/data.json"

# Paths to the output files
output_dir = f"{cwd}/out"
os.makedirs(output_dir, exist_ok=True)
model_onnx_path = f"{output_dir}/model.onnx"
compiled_model_path = f"{output_dir}/model.compiled"

pk_path = f"{output_dir}/model.pk"
vk_path = f"{output_dir}/model.vk"
proof_path = f"{output_dir}/model.pf"
settings_path = f"{output_dir}/settings.json"
witness_path = f"{output_dir}/witness.json"
comb_data_path = f"{output_dir}/comb_data.json"
data_commitment_path = f"{output_dir}/data_commitment.json"

## Columns selected by the user

In [12]:
# FIXME: this should be provided by users
selected_columns = ["x", "y"]

## User-defined Computation

A computation should be of type `TComputation`. For example, the following code snippet defines a computation that computes the sum of the private data.

```python
def computation(state: State, x: list[torch.Tensor]):
    out_0 = state.median(x[0])
    out_1 = state.median(x[1])
    return state.mean(torch.tensor([out_0, out_1]).reshape(1,-1,1))
```

FIXME: The following code snippet is entirely from the user. You MUST check
1. the code only performs zkstats-related operations.
2. the computation must not leak any information about the private data.

In [13]:
# This is just a dummy computation. Replace it with user's computation
import torch
from zkstats.computation import State

def computation(state: State, x: list[torch.Tensor]):
    out_0 = state.median(x[0])
    out_1 = state.median(x[1])
    return state.mean(torch.tensor([out_0, out_1]).reshape(1,-1,1))

Generate settings and setup with user's computation.

In [14]:
_, model = computation_to_model(computation)
prover_gen_settings(
    data_path,
    selected_columns,
    comb_data_path,
    model,
    model_onnx_path,
    "default",
    "resources",
    settings_path,
)

# Determine which srs to use with the logrows in the settings.json
with open(settings_path, "r") as f:
    settings = json.load(f)
logrows = int(settings["run_args"]["logrows"])
srs_path = f'~/.ezkl/srs/kzg{logrows}.srs'

/var/folders/t3/5psrvr1x0w1_6n9kx2n7d9700000gn/T/ipykernel_22219/2234676788.py:8: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return state.mean(torch.tensor([out_0, out_1]).reshape(1,-1,1))
/var/folders/t3/5psrvr1x0w1_6n9kx2n7d9700000gn/T/ipykernel_22219/2234676788.py:8: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  return state.mean(torch.tensor([out_0, out_1]).reshape(1,-1,1))


==== Generate & Calibrate Setting ====




 <------------- Numerical Fidelity Report (input_scale: 11, param_scale: 11, scale_input_multiplier: 10) ------------->

+----------------+----------------+-----------+----------------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| mean_error     | median_error   | max_error | min_error      | mean_abs_error | median_abs_error | max_abs_error | min_abs_error | mean_squared_error | mean_percent_error | mean_abs_percent_error |
+----------------+----------------+-----------+----------------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| -0.00004863739 | -0.00009727478 | 0         | -0.00009727478 | 0.00004863739  | 0.00009727478    | 0.00009727478 | 0             | 0.0000000047311914 | -0.000001957239    | 0.000001957239         |
+----------------+----------------+-----------+----------------+-------------

scale:  default
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":11,"param_scale":11,"scale_rebase_multiplier":10,"lookup_range":[-219952,192922],"logrows":19,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private","div_rebasing":false,"rebase_frac_zero_constants":false,"check_mode":"UNSAFE"},"num_rows":2624,"total_assignments":735,"total_const_size":152,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,11],"model_input_scales":[11,11],"module_sizes":{"kzg":[],"poseidon":[2624,[2]]},"required_lookups":[{"Floor":{"scale":2.0}},"ReLU",{"GreaterThan":{"a":0.0}}],"required_range_checks":[],"check_mode":"UNSAFE","version":"9.1.0","num_blinding_factors":null,"timestamp":1712764579432}


In [15]:
setup(model_onnx_path, compiled_model_path, settings_path, vk_path, pk_path)

==== setting up ezkl ====
Time setup: 89.43912076950073 seconds


Generate proof with your data and user's computation.

In [16]:
prover_gen_proof(
    model_onnx_path,
    comb_data_path,
    witness_path,
    compiled_model_path,
    settings_path,
    proof_path,
    pk_path,
)

==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 24.85009765625
==== Generating Proof ====
proof:  {'instances': [['4e29ced39e0837ccc54303617bee3278416698caa1099040db7586054c726819', '376d1c5357b7dca3dd77e4cd0cf36c71138900d41c115776af27ccb92d2f742e', '0100000000000000000000000000000000000000000000000000000000000000', 'cdc6000000000000000000000000000000000000000000000000000000000000']], 'proof': '0x027b20205bcca52008ce7af7dbc9c1e0828a3a1a84f7cc330e78a074176100732202e716004915bb4a17d143678fc821dad3d642d0971074016fbde229f77f20157b2b28784f04fba0f796ebe0e51d6de1c6780077d69509d0821d875cc9bd6322ac237ee28a4c289476a4ca1e1c662fbdd4ec166c82a4ef305d18d4c0ce7f9529e988d0efb45efec5f723bdeb9df37f9cc6249d88d35c1ac335b41f855426a6225026ba19763918961cc971bc334d3d230f7a597edc6ecb97fa9803a12da34e2247cdd4e037b029a338562b5c1f38c87104f1007d8ffd3b69eb840dd124ea0803567a5804195c19927d448b269179300893fc7c0c04c69820e5550d9ee89622049a45a972e6d509314746c8beff7f55c7b16b31076ef48190e70d63b9f66f3826

Verify the proof to ensure it is correct

In [17]:
verifier_verify(proof_path, settings_path, vk_path, selected_columns, data_commitment_path)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/mhchia/projects/work/pse/demo-next/public/assets/out/data_commitment.json'

Print the file paths. You should share the following files back to the user for them to verify the proof. You **SHOULD NOT** share more files otherwise data might be leaked.

In [ ]:
print("Model onnx:\t\t", model_onnx_path)
print("Settings:\t\t", settings_path)
print("Proof:\t\t\t", proof_path)
print("Verification key:\t", vk_path)
print("Srs path:\t\t", srs_path)

Model onnx:		 /Users/mhchia/projects/work/pse/demo-next/public/assets/out/model.onnx
Settings:		 /Users/mhchia/projects/work/pse/demo-next/public/assets/out/settings.json
Proof:			 /Users/mhchia/projects/work/pse/demo-next/public/assets/out/model.pf
Verification key:	 /Users/mhchia/projects/work/pse/demo-next/public/assets/out/model.vk
Srs path:		 ~/.ezkl/srs/kzg19.srs
